In [2]:
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
import glob
from model import Deeplabv3

# Generates labels using most basic setup.  Supports various image sizes.  Returns image labels in same format
# as original image.  Normalization matches MobileNetV2
image_dir = "./images"
for img_path in glob.glob(image_dir+"/*"):
    trained_image_width=512 
    self.mean_subtraction_value=127.5
    image = np.array(Image.open(img_path))

    # resize to max dimension of images from training dataset
    w, h, _ = image.shape
    ratio = float(trained_image_width) / np.max([w, h])
    resized_image = np.array(Image.fromarray(image.astype('uint8')).resize((int(ratio * h), int(ratio * w))))

    # apply normalization for trained dataset images
    resized_image = (resized_image / self.mean_subtraction_value) - 1.

    # pad array to square image to match training images
    pad_x = int(trained_image_width - resized_image.shape[0])
    pad_y = int(trained_image_width - resized_image.shape[1])
    resized_image = np.pad(resized_image, ((0, pad_x), (0, pad_y), (0, 0)), mode='constant')

    # make prediction
    deeplab_model = Deeplabv3()
    res = deeplab_model.predict(np.expand_dims(resized_image, 0))
    mask = np.argmax(res.squeeze(), -1) == 15  # region of person => 1, background => 0

    # remove padding and resize back to original image
    if pad_x > 0:
        mask = mask[:-pad_x]
    if pad_y > 0:
        mask = mask[:, :-pad_y]
    mask = np.array(Image.fromarray(mask.astype('uint8')).resize((h, w)))
    
    with_mask = image * np.stack([mask, mask, mask], 2)

#     plt.imshow(mask)
#     plt.show()
    plt.imshow(image)
    plt.show()
    plt.imshow(with_mask)
    plt.show()

In [42]:
from img2vec_keras import Img2Vec
img2vec = Img2Vec()
x = img2vec.get_vec(img_path)
x.shape

(2048,)